## Импорт библиотек

In [3]:
import cv2
import numpy as np
import time

Импортируем необходимые библиотеки: OpenCV для работы с видео и изображениями, NumPy для численных операций, time для измерения времени выполнения.

## Инициализация объекта захвата видео

In [4]:
video = cv2.VideoCapture('crowd.mp4')

## Загрузка меток классов COCO

In [5]:
with open('coco.names') as f:
    labels = [line.strip() for line in f]

Загружаем названия классов из файла coco.names и сохраняем их в списке labels

## Загрузка YOLO

In [6]:
network = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

Загружаем yolov3 с использованием весов и конфигурационного файла

## Получение имен выходных слоев

In [ ]:
layers_names_all = network.getLayerNames()
layers_names_output = [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]

In [ ]:
#print()
#print(layers_names_output)

Получаем имена всех слоев сети и выделяем имена только выходных слоев, которые нам нужны для предсказаний.

Устанавливаем минимальную вероятность для устранения слабых предсказаний и порог для фильтрации слабых bounding boxes, также генерируем случайные цвета для представления каждого класса.

In [8]:
probability_minimum = 0.5
threshold = 0.3
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [ ]:
#print(colours.shape)
#print(colours[0])

## Инициализация переменных

In [10]:
f = 0
t = 0
writer = None
w, h = None, None

 Инициализируем переменные для подсчета кадров, времени, объекта записи видео, а также ширины и высоты кадров.

## Цикл обработки кадров

In [ ]:
while True:
    ret, frame = video.read()
    if not ret:
        break

    if w is None or h is None:
        h, w = frame.shape[:2] #Читаем кадры из видео в цикле. Если кадр не был прочитан (например, конец видео), выходим из цикла. Устанавливаем ширину и высоту кадров, если они еще не были инициализированы.

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    network.setInput(blob) #Создаем blob из текущего кадра и устанавливаем его в качестве входного для сети.

    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    f += 1
    t += end - start

    print(f'Frame number {f} took {end - start:.5f} seconds') #Выполняем прямой проход через сеть и измеряем время, затраченное на обработку кадра. Увеличиваем счетчики кадров и времени.

    bounding_boxes = []
    confidences = []
    classIDs = []

    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]

            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                classIDs.append(class_current) #Извлекаем bounding boxes, уверенности и классы для каждого обнаруженного объекта. Фильтруем слабые предсказания на основе минимальной уверенности.

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold) #Применяем non-maximum suppression для удаления лишних bounding boxes.

    if len(results) > 0:
        for i in results.flatten():
            if classIDs[i] == 0:
                x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
                box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
                colour_box_current = colours[classIDs[i]].tolist()

                cv2.rectangle(frame, (x_min, y_min), (x_min + box_width, y_min + box_height), colour_box_current, 2)
                text_box_current = f'{labels[0]}: {confidences[i]:.4f}'
                cv2.putText(frame, text_box_current, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2) #Рисуем bounding boxes и метки на кадре только для объектов класса "человек" (class ID 0).

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter('submit.mp4', fourcc, 30, (frame.shape[1], frame.shape[0]), True)

    writer.write(frame) #Создаем объект VideoWriter для записи выходного видео, если он еще не был создан. Записываем текущий кадр в выходное видео.

Освобождаем ресурсы и выводим общую информацию о количестве обработанных кадров, затраченном времени и fps.

In [ ]:
video.release()
writer.release()

print(f'Total number of frames: {f}')
print(f'Total amount of time: {t:.5f} seconds')
print(f'FPS: {round(f / t, 1)}')